<a href="https://colab.research.google.com/github/divyam123-EECS-Physics/NetLoadForecastingScripts/blob/main/Universal_PyTorch_Manual_Tuning_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [262]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Imports and Utilities

In [263]:

import pandas as pd
!pip install solarforecastarbiter
!pip install TimezoneFinder

import numpy as np
import matplotlib.pyplot as plt
from timezonefinder import TimezoneFinder
import solarforecastarbiter.metrics
from solarforecastarbiter.metrics.probabilistic import continuous_ranked_probability_score,crps_skill_score
import solarforecastarbiter.reference_forecasts
from solarforecastarbiter.reference_forecasts.persistence import persistence_probabilistic_timeofday, persistence_probabilistic
# from gluonts.mx.model.deep_npts import DeepNPTSEstimator
import torch
import scipy as sp

import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split


In [264]:
from timezonefinder import TimezoneFinder
states = {
    'hi': {
        "state_location":'hawaii',
        "lat":21.446911,
        "lon":-158.188736,
        "elevation":4.0,
        "fresh_data_time":'12',
        "reference_id":"c639b1f3-eb8f-11ed-802e-aec5a60999dc",
        "name":"Waianae, HI Substation",    },
    'or': {
        "state_location":'oregon',
        "lat":45.114559,
        "lon":-123.204903,
        "elevation":50.0,
        "fresh_data_time":'12',
        "reference_id":"5ebb4527-edbd-11ed-bf8d-128dcacebd72",
        "name":"Amity, OR Substation",
    },
    'tx': {
         "state_location":'texas',
        "lat":29.424122,
        "lon":-98.493629,
        "elevation":200.0,
        "fresh_data_time":'06',
        "reference_id":"8568f10f-eb8f-11ed-a556-128dcacebd72",
        "name":"San Antonio, TX Substation",

    },
    'ga': {
        "state_location":'georgia',
        "lat":31.044241,
        "lon":-84.879128,
        "elevation":0.0,
        "fresh_data_time":'06',
        "reference_id":"90c2a42c-f0ad-11ed-94b4-5edf5e2b3336",
        "name":"Donalsonville, GA Substation"
    },

}

tf = TimezoneFinder(in_memory=True)
for state in states:
  local_time_zone = tf.timezone_at(lng = states[state]['lon'], lat = states[state]['lat'])
  states[state]['local_time_zone'] = local_time_zone


tx_lon = -98.493629
tx_lat = 29.424122
ga_lon = -84.879128
ga_lat = 31.044241
or_lon = -123.204903
or_lat = 45.114559
hi_lat = 21.446911
hi_lon = -158.188736



from solarforecastarbiter.io import api
def upload_function(data_df, state = 'hi'):
    email = 'nlf2023-coral@forecastarbiter.com'
    pwd =  'T3&b@O*KiGf$a@!@3LBv'
    token = api.request_cli_access_token(email, pwd)
    session = api.APISession(token, base_url="https://api.forecastarbiter.com")
    if state == 'hi':

        hawaii_forecast_ids = [
           ["60534840-0044-11ee-8096-22bbbfba48a8", "p0"],
           ["60551d2e-0044-11ee-aea4-22bbbfba48a8", "p10"],
           ["60577a7c-0044-11ee-9d93-22bbbfba48a8", "p20"],
           ["6059b4ae-0044-11ee-b9f7-22bbbfba48a8", "p30"],
           ["605c7e09-0044-11ee-88c5-22bbbfba48a8", "p40"],
           ["605f5aad-0044-11ee-a68f-22bbbfba48a8", "p50"],
           ["6061a141-0044-11ee-b645-22bbbfba48a8", "p60"],
           ["6063f5d3-0044-11ee-a725-22bbbfba48a8", "p70"],
           ["606656a8-0044-11ee-aca6-22bbbfba48a8", "p80"],
           ["6069aa58-0044-11ee-b4a8-22bbbfba48a8", "p90"],
           ["606c387a-0044-11ee-92ea-22bbbfba48a8", "p100"],
        ]
        for uuid, col in hawaii_forecast_ids:
            upload_df = data_df[col]
            upload_df = upload_df.rename('value')
            session.post_probabilistic_forecast_constant_value_values(uuid, upload_df)

        return
    if state == 'tx':
        tx_forecast_ids = [
           ["51936f7f-0044-11ee-8006-22bbbfba48a8", "p0"],
           ["5195a4a9-0044-11ee-9abe-22bbbfba48a8", "p10"],
           ["5199b969-0044-11ee-8c5b-22bbbfba48a8", "p20"],
           ["519be6af-0044-11ee-aa1b-22bbbfba48a8", "p30"],
           ["519e4691-0044-11ee-ace3-22bbbfba48a8", "p40"],
           ["51a07836-0044-11ee-b70a-22bbbfba48a8", "p50"],
           ["51a2b229-0044-11ee-9369-22bbbfba48a8", "p60"],
           ["51a54811-0044-11ee-9b1c-22bbbfba48a8", "p70"],
           ["51a78587-0044-11ee-ac5f-22bbbfba48a8", "p80"],
           ["51a96889-0044-11ee-95de-22bbbfba48a8", "p90"],
           ["51ac716f-0044-11ee-96a0-22bbbfba48a8", "p100"],
        ]
        for uuid, col in tx_forecast_ids:
            upload_df = data_df[col]
            upload_df = upload_df.rename('value')
            session.post_probabilistic_forecast_constant_value_values(uuid, upload_df)

        return
    if state == 'or':

        or_forecast_ids = [
           ["58ea38ee-0044-11ee-abf9-aa678857fb1c", "p0"],
           ["58ecd0ac-0044-11ee-96ef-aa678857fb1c", "p10"],
           ["58efa6af-0044-11ee-a87e-aa678857fb1c", "p20"],
           ["58f2856f-0044-11ee-94d6-aa678857fb1c", "p30"],
           ["58f4d11c-0044-11ee-80ca-aa678857fb1c", "p40"],
           ["58f792d8-0044-11ee-869d-aa678857fb1c", "p50"],
           ["58fa355e-0044-11ee-881a-aa678857fb1c", "p60"],
           ["58fc98a1-0044-11ee-b2cb-aa678857fb1c", "p70"],
           ["58ff236c-0044-11ee-ba4c-aa678857fb1c", "p80"],
           ["5901b51f-0044-11ee-aae2-aa678857fb1c", "p90"],
           ["59045e2b-0044-11ee-8914-aa678857fb1c", "p100"],
        ]
        for uuid, col in or_forecast_ids:
            upload_df = data_df[col]
            upload_df = upload_df.rename('value')
            session.post_probabilistic_forecast_constant_value_values(uuid, upload_df)

        return
    if state == 'ga':

        ga_forecast_ids = [
           ["493ba331-0044-11ee-936c-aa678857fb1c", "p0"],
           ["493e3bce-0044-11ee-bdf4-aa678857fb1c", "p10"],
           ["4940a921-0044-11ee-8e27-aa678857fb1c", "p20"],
           ["49431d34-0044-11ee-a476-aa678857fb1c", "p30"],
           ["4945f80a-0044-11ee-8674-aa678857fb1c", "p40"],
           ["49489e6d-0044-11ee-8ce3-aa678857fb1c", "p50"],
           ["494b350e-0044-11ee-b5bc-aa678857fb1c", "p60"],
           ["494fe072-0044-11ee-bb2d-aa678857fb1c", "p70"],
           ["4952cab3-0044-11ee-afa5-aa678857fb1c", "p80"],
           ["4955940c-0044-11ee-a0c7-aa678857fb1c", "p90"],
           ["495827d0-0044-11ee-83b0-aa678857fb1c", "p100"],
        ]

        for uuid, col in ga_forecast_ids:
            upload_df = data_df[col]
            upload_df = upload_df.rename('value')
            print(upload_df)
            session.post_probabilistic_forecast_constant_value_values(uuid, upload_df)

        return


def get_reference_forecasts(location, start_date, end_date):
    # Access related parameters for Net Load Forecast Site
    email = 'nlf2023-coral@forecastarbiter.com'
    pwd =  'T3&b@O*KiGf$a@!@3LBv'
    start_datetime = start_date + ' 01:00:00'
    end_datetime = end_date + ' 00:00:00'
    #Reference Probablistic Persistance Ensemble Forecast
    locations = {'4a3de81c-0043-11ee-a474-aa678857fb1c':['Pacific/Honolulu','Waianae_Hawaii'],
                '49cdf54e-0043-11ee-a581-aa678857fb1c':['America/Los_Angeles','Amity_OR'],
                '496055a3-0043-11ee-ada4-aa678857fb1c':['America/Chicago','SanAntonio_TX'],
                '3b6b45e3-0043-11ee-bfe9-aa678857fb1c':['America/New_York','Donalsonville_GA']}
    locations_uuids = {
        'hi':'4a3de81c-0043-11ee-a474-aa678857fb1c',
        'or':'49cdf54e-0043-11ee-a581-aa678857fb1c',
        'tx':'496055a3-0043-11ee-ada4-aa678857fb1c',
        'ga':'3b6b45e3-0043-11ee-bfe9-aa678857fb1c'
    }
    # for uuid in locations.keys():
    uuid = locations_uuids[location]
    tmz=locations[uuid][0]

    token = api.request_cli_access_token(email, pwd)
    session = api.APISession(token, base_url="https://api.forecastarbiter.com")

    session.get_probabilistic_forecast(uuid)

    start = pd.Timestamp(start_datetime).tz_localize(tmz)
    end = pd.Timestamp(end_datetime).tz_localize(tmz)
    fdf = session.get_probabilistic_forecast_values(uuid, start, end)
    fdf = fdf.reset_index('timestamp')
    fdf = fdf.rename(columns = {'timestamp':'time'})
    fdf.time = fdf.time.dt.tz_convert(tmz)
    fdf.time = fdf.time.dt.tz_localize(None)
    return fdf

def crps_and_crpss_scores(obs, predictions, start_date, end_date, location):

    fx_prob = torch.tensor([i for i in range(0,110,10)]).unsqueeze(0).repeat(24,1).numpy()
    reference_df = get_reference_forecasts(location, start_date, end_date)
    reference_val = reference_df.drop(columns = 'time').values

    crps = continuous_ranked_probability_score(obs, predictions, fx_prob)
    crpss = crps_skill_score(obs, predictions,  fx_prob, reference_val, fx_prob)

    return crps, crpss, reference_df




# def make_reference_forecasts(start_date, end_date, location)

#     location_meta_data = {
#         'tx': ['San Antonio, TX Substation',states['tx']['lat'],]
#     }


#     name = 'San Antonio, TX Substation'
#     latitude = lat
#     longitude = lon
#     elevation = 4.0
#     timezone = local_time_zone
#     site_id = 'c639b1f3-eb8f-11ed-802e-aec5a60999dc'
#     site = Site(name = name, latitude = latitude, longitude = longitude, elevation = elevation,
#                 timezone = timezone, site_id = site_id)

#     variable = 'net_load'
#     interval_value_type = 'interval_mean'
#     interval_length = pd.Timedelta(value = 1, unit = 'hour')
#     interval_label = 'ending'
#     site = site
#     uncertainty = 0
#     obs = Observation(name = name, variable = variable,
#                       interval_value_type = interval_value_type,
#                       interval_length = interval_length,
#                       interval_label = interval_label,
#                       site = site, uncertainty = uncertainty)

#     observation  = obs
#     data_start  = start_time
#     data_end = end_time
#     forecast_start = target_start_time
#     forecast_end  = target_end_time
#     interval_length = interval_length
#     interval_label = 'ending'
#     load_data = load_data
#     axis = 'x'
#     constant_values = np.array([x/10 for x in range(1,10)])


#     baseline = persistence_probabilistic(observation = obs,
#                                         data_start = data_start,
#                                         data_end = data_end,
#                                         forecast_start = forecast_start,
#                                         forecast_end = forecast_end,
#                                         interval_length = interval_length,
#                                         interval_label = interval_label,
#                                         load_data = load_data,
#                                         axis = 'y',
#                                         constant_values = constant_values)


# Feature engineering


In [265]:

state_location = 'tx'
training_data = pd.read_csv('/content/drive/MyDrive/net_load_forecasting/training_datasets/{}_historical.csv'.format(states[state_location]['state_location']))
total_data = pd.read_csv('/content/drive/MyDrive/net_load_forecasting/training_datasets/{}_2022-01-01-2023-06-28_total.csv'.format(states[state_location]['state_location']))
test_data = pd.read_csv('/content/drive/MyDrive/net_load_forecasting/test_datasets/{}_2023-07-03_test.csv'.format(states[state_location]['state_location']))



In [266]:
training_data.rename(columns = {'Unnamed: 0':'time'}, inplace = True)
test_data.rename(columns = {'Unnamed: 0':'time'}, inplace = True)

training_data.drop(columns = 'quality_flag', inplace = True)
test_data.drop(columns = 'quality_flag', inplace = True)

In [267]:
test_data

,time,value,temperature_2m,relativehumidity_2m,dewpoint_2m,apparent_temperature,precipitation,cloudcover,cloudcover_low,cloudcover_mid,cloudcover_high,shortwave_radiation,direct_radiation,diffuse_radiation,direct_normal_irradiance,sunset,sunrise
0,2023-06-01 01:00:00,0.405947,23.0,84,20.2,25.1,0.0,27,27,0,0,0.0,0.0,0.0,0.0,2023-06-01T20:29,2023-06-01T06:34
1,2023-06-01 02:00:00,0.358691,22.5,84,19.7,25.2,0.0,1,1,0,0,0.0,0.0,0.0,0.0,2023-06-01T20:29,2023-06-01T06:34
2,2023-06-01 03:00:00,0.335408,22.0,85,19.4,24.6,0.0,100,100,0,0,0.0,0.0,0.0,0.0,2023-06-01T20:29,2023-06-01T06:34
3,2023-06-01 04:00:00,0.341229,21.9,86,19.4,24.4,0.0,80,80,0,0,0.0,0.0,0.0,0.0,2023-06-01T20:29,2023-06-01T06:34
4,2023-06-01 05:00:00,0.379495,22.1,90,20.3,24.9,0.0,99,99,0,0,0.0,0.0,0.0,0.0,2023-06-01T20:29,2023-06-01T06:34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
834,2023-07-05 19:00:00,NaN,33.8,40,18.3,31.7,0.0,44,42,33,0,315.0,236.0,79.0,554.7,2023-07-05T20:37,2023-07-05T06:39
835,2023-07-05 20:00:00,NaN,32.5,44,18.7,31.4,0.0,0,0,0,0,110.0,69.0,41.0,314.6,2023-07-05T20:37,2023-07-05T06:39
836,2023-07-05 21:00:00,NaN,31.3,50,19.6,30.6,0.0,0,0,0,0,14.0,8.0,6.0,85.6,2023-07-05T20:37,2023-07-05T06:39
837,2023-07-05 22:00:00,NaN,29.6,57,20.2,29.4,0.0,0,0,0,0,0.0,0.0,0.0,0.0,2023-07-05T20:37,2023-07-05T06:39


In [268]:
training_data.time = pd.to_datetime(training_data.time)
test_data.time = pd.to_datetime(test_data.time)

In [269]:
pd.to_datetime('2023-')

Timestamp('2023-01-01 00:00:00')

In [270]:
test_data[test_data.value.isna()] # forecasting for the 4th

,time,value,temperature_2m,relativehumidity_2m,dewpoint_2m,apparent_temperature,precipitation,cloudcover,cloudcover_low,cloudcover_mid,cloudcover_high,shortwave_radiation,direct_radiation,diffuse_radiation,direct_normal_irradiance,sunset,sunrise
704,2023-06-30 09:00:00,NaN,27.8,77,23.3,31.2,0.0,100,100,0,10,180.0,10.0,170.0,26.7,2023-06-30T20:37,2023-06-30T06:37
705,2023-06-30 10:00:00,NaN,29.2,73,23.9,32.9,0.0,100,100,0,19,366.0,208.0,158.0,365.3,2023-06-30T20:37,2023-06-30T06:37
706,2023-06-30 11:00:00,NaN,31.6,62,23.4,35.0,0.0,38,29,0,38,573.0,362.0,211.0,490.0,2023-06-30T20:37,2023-06-30T06:37
707,2023-06-30 12:00:00,NaN,32.9,60,24.1,38.9,0.0,40,34,0,40,943.0,806.0,137.0,925.2,2023-06-30T20:37,2023-06-30T06:37
708,2023-06-30 13:00:00,NaN,34.6,51,23.0,40.0,0.0,26,21,0,26,990.0,646.0,344.0,674.8,2023-06-30T20:37,2023-06-30T06:37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
834,2023-07-05 19:00:00,NaN,33.8,40,18.3,31.7,0.0,44,42,33,0,315.0,236.0,79.0,554.7,2023-07-05T20:37,2023-07-05T06:39
835,2023-07-05 20:00:00,NaN,32.5,44,18.7,31.4,0.0,0,0,0,0,110.0,69.0,41.0,314.6,2023-07-05T20:37,2023-07-05T06:39
836,2023-07-05 21:00:00,NaN,31.3,50,19.6,30.6,0.0,0,0,0,0,14.0,8.0,6.0,85.6,2023-07-05T20:37,2023-07-05T06:39
837,2023-07-05 22:00:00,NaN,29.6,57,20.2,29.4,0.0,0,0,0,0,0.0,0.0,0.0,0.0,2023-07-05T20:37,2023-07-05T06:39


In [271]:
training_data['hours'] = np.sin((training_data.time.dt.hour - 12) / 24)
test_data['hours'] = np.sin((test_data.time.dt.hour - 12) / 24)

In [272]:

test_data

,time,value,temperature_2m,relativehumidity_2m,dewpoint_2m,apparent_temperature,precipitation,cloudcover,cloudcover_low,cloudcover_mid,cloudcover_high,shortwave_radiation,direct_radiation,diffuse_radiation,direct_normal_irradiance,sunset,sunrise,hours
0,2023-06-01 01:00:00,0.405947,23.0,84,20.2,25.1,0.0,27,27,0,0,0.0,0.0,0.0,0.0,2023-06-01T20:29,2023-06-01T06:34,-0.442454
1,2023-06-01 02:00:00,0.358691,22.5,84,19.7,25.2,0.0,1,1,0,0,0.0,0.0,0.0,0.0,2023-06-01T20:29,2023-06-01T06:34,-0.404715
2,2023-06-01 03:00:00,0.335408,22.0,85,19.4,24.6,0.0,100,100,0,0,0.0,0.0,0.0,0.0,2023-06-01T20:29,2023-06-01T06:34,-0.366273
3,2023-06-01 04:00:00,0.341229,21.9,86,19.4,24.4,0.0,80,80,0,0,0.0,0.0,0.0,0.0,2023-06-01T20:29,2023-06-01T06:34,-0.327195
4,2023-06-01 05:00:00,0.379495,22.1,90,20.3,24.9,0.0,99,99,0,0,0.0,0.0,0.0,0.0,2023-06-01T20:29,2023-06-01T06:34,-0.287549
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
834,2023-07-05 19:00:00,NaN,33.8,40,18.3,31.7,0.0,44,42,33,0,315.0,236.0,79.0,554.7,2023-07-05T20:37,2023-07-05T06:39,0.287549
835,2023-07-05 20:00:00,NaN,32.5,44,18.7,31.4,0.0,0,0,0,0,110.0,69.0,41.0,314.6,2023-07-05T20:37,2023-07-05T06:39,0.327195
836,2023-07-05 21:00:00,NaN,31.3,50,19.6,30.6,0.0,0,0,0,0,14.0,8.0,6.0,85.6,2023-07-05T20:37,2023-07-05T06:39,0.366273
837,2023-07-05 22:00:00,NaN,29.6,57,20.2,29.4,0.0,0,0,0,0,0.0,0.0,0.0,0.0,2023-07-05T20:37,2023-07-05T06:39,0.404715


In [273]:
test_data['sunrise'] = pd.to_datetime(test_data.sunrise)
test_data['sunset'] = pd.to_datetime(test_data.sunset)

test_data.sunrise = np.sin((test_data.sunrise.dt.hour + (test_data.sunrise.dt.minute / 60) - 12) / 24)
test_data.sunset = np.sin((test_data.sunset.dt.hour + (test_data.sunset.dt.minute / 60) - 12) / 24)

In [274]:
test_data

,time,value,temperature_2m,relativehumidity_2m,dewpoint_2m,apparent_temperature,precipitation,cloudcover,cloudcover_low,cloudcover_mid,cloudcover_high,shortwave_radiation,direct_radiation,diffuse_radiation,direct_normal_irradiance,sunset,sunrise,hours
0,2023-06-01 01:00:00,0.405947,23.0,84,20.2,25.1,0.0,27,27,0,0,0.0,0.0,0.0,0.0,0.346157,-0.224460,-0.442454
1,2023-06-01 02:00:00,0.358691,22.5,84,19.7,25.2,0.0,1,1,0,0,0.0,0.0,0.0,0.0,0.346157,-0.224460,-0.404715
2,2023-06-01 03:00:00,0.335408,22.0,85,19.4,24.6,0.0,100,100,0,0,0.0,0.0,0.0,0.0,0.346157,-0.224460,-0.366273
3,2023-06-01 04:00:00,0.341229,21.9,86,19.4,24.4,0.0,80,80,0,0,0.0,0.0,0.0,0.0,0.346157,-0.224460,-0.327195
4,2023-06-01 05:00:00,0.379495,22.1,90,20.3,24.9,0.0,99,99,0,0,0.0,0.0,0.0,0.0,0.346157,-0.224460,-0.287549
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
834,2023-07-05 19:00:00,NaN,33.8,40,18.3,31.7,0.0,44,42,33,0,315.0,236.0,79.0,554.7,0.351364,-0.221075,0.287549
835,2023-07-05 20:00:00,NaN,32.5,44,18.7,31.4,0.0,0,0,0,0,110.0,69.0,41.0,314.6,0.351364,-0.221075,0.327195
836,2023-07-05 21:00:00,NaN,31.3,50,19.6,30.6,0.0,0,0,0,0,14.0,8.0,6.0,85.6,0.351364,-0.221075,0.366273
837,2023-07-05 22:00:00,NaN,29.6,57,20.2,29.4,0.0,0,0,0,0,0.0,0.0,0.0,0.0,0.351364,-0.221075,0.404715


In [275]:
training_data['sunrise'] = pd.to_datetime(training_data.sunrise)
training_data['sunset'] = pd.to_datetime(training_data.sunset)

training_data.sunrise = np.sin((training_data.sunrise.dt.hour + (training_data.sunrise.dt.minute / 60) - 12) / 24)
training_data.sunset = np.sin((training_data.sunset.dt.hour + (training_data.sunset.dt.minute / 60) - 12) / 24)

In [276]:
training_data['month'] = training_data.time.dt.month
test_data['month'] = test_data.time.dt.month

In [277]:
pd.to_datetime('2023-07-04 01:00:00') - pd.to_datetime('2023-07-01 01:00:00')
# test_data[test_data.value.isna()]

Timedelta('3 days 00:00:00')

In [278]:
lookback_start = 5
lookback_dim = 10
lookback_end = lookback_dim + lookback_start - 1
# lookback_dim = lookback_end - lookback_start + 1

for i in range(lookback_start, lookback_end + 1):
  training_data['prev_{}'.format(i)] = training_data.shift(i * 24).value
  test_data['prev_{}'.format(i)] = test_data.shift(i * 24).value



In [279]:
test_data

,time,value,temperature_2m,relativehumidity_2m,dewpoint_2m,apparent_temperature,precipitation,cloudcover,cloudcover_low,cloudcover_mid,...,prev_5,prev_6,prev_7,prev_8,prev_9,prev_10,prev_11,prev_12,prev_13,prev_14
0,2023-06-01 01:00:00,0.405947,23.0,84,20.2,25.1,0.0,27,27,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-06-01 02:00:00,0.358691,22.5,84,19.7,25.2,0.0,1,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2023-06-01 03:00:00,0.335408,22.0,85,19.4,24.6,0.0,100,100,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2023-06-01 04:00:00,0.341229,21.9,86,19.4,24.4,0.0,80,80,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2023-06-01 05:00:00,0.379495,22.1,90,20.3,24.9,0.0,99,99,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
834,2023-07-05 19:00:00,NaN,33.8,40,18.3,31.7,0.0,44,42,33,...,NaN,0.800426,0.847165,0.891079,0.928308,0.919951,0.830740,0.857480,0.759970,0.878861
835,2023-07-05 20:00:00,NaN,32.5,44,18.7,31.4,0.0,0,0,0,...,NaN,0.794548,0.840249,0.882722,0.907619,0.896611,0.833333,0.834659,0.757204,0.824631
836,2023-07-05 21:00:00,NaN,31.3,50,19.6,30.6,0.0,0,0,0,...,NaN,0.756973,0.818061,0.866125,0.873617,0.877536,0.805095,0.801579,0.730290,0.743949
837,2023-07-05 22:00:00,NaN,29.6,57,20.2,29.4,0.0,0,0,0,...,NaN,0.721704,0.771899,0.823651,0.834371,0.835581,0.760950,0.777835,0.694675,0.703492


In [280]:
training_data.columns

Index(['time', 'value', 'temperature_2m', 'relativehumidity_2m', 'dewpoint_2m',
       'apparent_temperature', 'precipitation', 'cloudcover', 'cloudcover_low',
       'cloudcover_mid', 'cloudcover_high', 'shortwave_radiation',
       'direct_radiation', 'diffuse_radiation', 'direct_normal_irradiance',
       'sunset', 'sunrise', 'hours', 'month', 'prev_5', 'prev_6', 'prev_7',
       'prev_8', 'prev_9', 'prev_10', 'prev_11', 'prev_12', 'prev_13',
       'prev_14'],
      dtype='object')

In [281]:
training_data = training_data[(training_data['prev_{}'.format(lookback_end)].isna() == False)].reset_index().drop(columns = 'index')

In [282]:
training_data

,time,value,temperature_2m,relativehumidity_2m,dewpoint_2m,apparent_temperature,precipitation,cloudcover,cloudcover_low,cloudcover_mid,...,prev_5,prev_6,prev_7,prev_8,prev_9,prev_10,prev_11,prev_12,prev_13,prev_14
0,2022-01-15 01:00:00,0.290802,12.6,78,8.8,11.0,0.0,15,0,25,...,0.295816,0.298006,0.361918,0.357480,0.296277,0.317485,0.363935,0.402086,0.310281,0.420758
1,2022-01-15 02:00:00,0.278873,13.4,62,6.3,9.3,0.0,13,0,21,...,0.299504,0.286019,0.353446,0.341229,0.287229,0.314488,0.356213,0.406178,0.292819,0.392404
2,2022-01-15 03:00:00,0.260489,14.0,35,-1.3,7.4,0.0,30,0,50,...,0.319675,0.278008,0.335177,0.337252,0.268903,0.315526,0.359440,0.403124,0.286249,0.379380
3,2022-01-15 04:00:00,0.251671,12.9,41,-0.1,5.9,0.0,30,0,48,...,0.313739,0.288382,0.313624,0.347165,0.266482,0.314949,0.360592,0.416321,0.286077,0.356155
4,2022-01-15 05:00:00,0.265848,11.3,47,0.3,4.1,0.0,54,0,74,...,0.342324,0.274205,0.307400,0.395920,0.308725,0.334140,0.373329,0.432284,0.290975,0.339557
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12004,2023-05-30 07:00:00,0.306132,19.1,96,18.4,21.8,0.0,0,0,0,...,0.381051,0.373559,0.385892,0.347914,0.282388,0.300542,0.405775,0.355175,0.345090,0.352812
12005,2023-05-30 08:00:00,0.306478,21.1,92,19.8,24.3,0.0,0,0,0,...,0.372291,0.352178,0.366471,0.340018,0.283310,0.302732,0.397130,0.358691,0.331143,0.349412
12006,2023-05-30 09:00:00,0.300599,23.4,81,19.9,26.8,0.0,0,0,0,...,0.361342,0.355406,0.350046,0.337079,0.308437,0.324919,0.397822,0.357250,0.325496,0.347971
12007,2023-05-30 10:00:00,0.344456,25.0,72,19.7,28.4,0.0,0,0,0,...,0.363013,0.370562,0.376671,0.325092,0.293799,0.349182,0.386814,0.364396,0.330048,0.336503


In [283]:
new_df = training_data[(training_data.value.isna() == False) &
                   (training_data['prev_{}'.format(lookback_end)].isna() == False)]#.drop(columns = 'time')

new_df = new_df[((new_df['time'] <= '2022-10-01 00:00:00') & (new_df['time'] >= '2022-05-01 00:00:00'))].drop(columns = 'time')


In [284]:
new_df

,value,temperature_2m,relativehumidity_2m,dewpoint_2m,apparent_temperature,precipitation,cloudcover,cloudcover_low,cloudcover_mid,cloudcover_high,...,prev_5,prev_6,prev_7,prev_8,prev_9,prev_10,prev_11,prev_12,prev_13,prev_14
2542,0.479541,20.7,95,19.8,22.6,0.0,26,28,2,0,...,0.321692,0.444444,0.441966,0.424331,0.416148,0.445021,0.343303,0.364569,0.439776,0.492739
2543,0.441793,20.5,94,19.6,22.4,0.0,25,24,4,2,...,0.277144,0.398167,0.398225,0.381339,0.371657,0.395747,0.316102,0.326360,0.389696,0.440929
2544,0.385546,21.8,93,20.6,24.6,0.0,38,23,3,51,...,0.258011,0.354714,0.372003,0.354657,0.345897,0.348548,0.301579,0.294663,0.358575,0.402490
2545,0.364281,21.3,95,20.6,24.6,0.0,38,27,4,39,...,0.256800,0.363877,0.347107,0.330855,0.336503,0.344917,0.290341,0.277374,0.340998,0.361111
2546,0.350795,21.1,96,20.5,24.2,0.0,48,29,0,72,...,0.256627,0.340018,0.327570,0.315353,0.324689,0.337483,0.291090,0.275876,0.318926,0.348317
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6210,0.648283,28.1,29,8.5,26.0,0.0,0,0,0,0,...,0.869640,0.752824,0.786595,0.807054,0.800599,0.836330,0.870908,0.853331,0.780486,0.765387
6211,0.626729,26.7,32,8.4,24.7,0.0,0,0,0,0,...,0.791263,0.715134,0.724758,0.761123,0.746024,0.812529,0.824516,0.821750,0.732711,0.740664
6212,0.595609,25.5,34,8.5,23.6,0.0,0,0,0,0,...,0.721012,0.665572,0.668165,0.703492,0.684013,0.747868,0.789304,0.773744,0.688163,0.713462
6213,0.528469,24.4,36,8.5,22.6,0.0,0,0,0,0,...,0.646323,0.631455,0.628285,0.610708,0.598317,0.669606,0.715422,0.692658,0.652374,0.684013


In [285]:


val_df = test_data[(test_data.value.isna() == False) &
                   (test_data['prev_{}'.format(lookback_end)].isna() == False)].drop(columns = 'time')
test_df_new = test_data[test_data.value.isna() == True][-24-23:-23].reset_index().drop(columns = ['index','time'])

In [286]:
test_data[(test_data.value.isna() == False) & (test_data.prev_8.isna() == False)]

,time,value,temperature_2m,relativehumidity_2m,dewpoint_2m,apparent_temperature,precipitation,cloudcover,cloudcover_low,cloudcover_mid,...,prev_5,prev_6,prev_7,prev_8,prev_9,prev_10,prev_11,prev_12,prev_13,prev_14
192,2023-06-09 01:00:00,0.454933,24.8,80,21.1,28.6,0.0,0,0,0,...,0.396900,0.373963,0.417128,0.405947,NaN,NaN,NaN,NaN,NaN,NaN
193,2023-06-09 02:00:00,0.405256,24.3,82,21.0,28.3,0.0,0,0,0,...,0.374366,0.344283,0.383875,0.358691,NaN,NaN,NaN,NaN,NaN,NaN
194,2023-06-09 03:00:00,0.362379,23.6,82,20.3,27.3,0.0,0,0,0,...,0.347165,0.309763,0.351199,0.335408,NaN,NaN,NaN,NaN,NaN,NaN
195,2023-06-09 04:00:00,0.338405,23.7,78,19.6,27.1,0.0,0,0,0,...,0.321980,0.277317,0.335177,0.341229,NaN,NaN,NaN,NaN,NaN,NaN
196,2023-06-09 05:00:00,0.342958,23.5,81,20.0,26.9,0.0,0,0,0,...,0.313163,0.264408,0.386353,0.379495,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
699,2023-06-30 04:00:00,0.459832,27.0,78,22.8,30.1,0.0,100,100,0,...,0.449113,0.446577,0.410212,0.416090,0.457527,0.454242,0.481731,0.480233,0.483287,0.453723
700,2023-06-30 05:00:00,0.440756,27.1,80,23.3,30.7,0.0,100,100,0,...,0.420067,0.407561,0.392174,0.410270,0.442312,0.442773,0.454703,0.439661,0.479772,0.466920
701,2023-06-30 06:00:00,0.457296,27.1,79,23.1,30.6,0.0,94,94,0,...,0.404046,0.410270,0.393557,0.420067,0.448536,0.476314,0.436030,0.425945,0.470436,0.459083
702,2023-06-30 07:00:00,0.449746,26.6,83,23.5,30.8,0.0,100,100,0,...,0.400184,0.397130,0.414477,0.406293,0.460120,0.484843,0.422833,0.424504,0.453723,0.466632


In [287]:
val_df=val_df[new_df.columns]
test_df_new=test_df_new[new_df.columns]

# Train Val Split

In [288]:
X_train = new_df[[col for col in new_df.columns if col != 'value']]
y_train = new_df.value
X_train = torch.from_numpy(X_train.values).to(torch.float64)
y_train = torch.from_numpy(y_train.values).to(torch.float64)


In [289]:
X_val = val_df[[col for col in val_df.columns if col != 'value']]
y_val = val_df.value
X_val = torch.from_numpy(X_val.values).to(torch.float64)
y_val = torch.from_numpy(y_val.values).to(torch.float64)

# Defining models

In [298]:
class TrendsModel(nn.Module):
  def __init__(self, input_dim, hidden_dim, output_dim):
    super(TrendsModel, self).__init__()
    self.fc1 = nn.Linear(input_dim, hidden_dim).to(torch.float64)
    # self.fc2 = nn.Linear(hidden_dim, 1).to(torch.float64)
    self.fc2 = nn.Linear(hidden_dim, hidden_dim).to(torch.float64)
    self.fc3 = nn.Linear(hidden_dim, 1).to(torch.float64)
    # self.fc4 = nn.Linear(hidden_dim, hidden_dim).to(torch.float64)
    # self.fc5 = nn.Linear(hidden_dim, hidden_dim).to(torch.float64)
    # self.fc6 = nn.Linear(hidden_dim, hidden_dim).to(torch.float64)
    # self.fc7 = nn.Linear(hidden_dim, 1).to(torch.float64)
    self.relu = nn.ReLU().to(torch.float64)
    # self.softplus = nn.Softplus().to(torch.float64)
    self.softmax = nn.Softmax().to(torch.float64)
  def forward(self, x):
    x = self.fc1(x)
    x = self.relu(x)
    x = self.fc2(x)
    x = self.relu(x)
    x = self.fc3(x)
    # x = self.relu(x)
    # x = self.fc4(x)
    # x = self.relu(x)
    # x = self.fc5(x)
    # x = self.relu(x)
    # x = self.fc6(x)
    # x = self.relu(x)
    # x = self.fc7(x)
    return x

class FeaturesModel(nn.Module):
    def __init__(self, input_dim, hidden_dim,output_dim):
        super(FeaturesModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim).to(torch.float64)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim).to(torch.float64)
        self.fc3 = nn.Linear(hidden_dim, hidden_dim).to(torch.float64)
        self.fc4 = nn.Linear(hidden_dim, hidden_dim).to(torch.float64)
        self.fc5 = nn.Linear(hidden_dim, hidden_dim).to(torch.float64)
        self.fc6 = nn.Linear(hidden_dim, hidden_dim).to(torch.float64)
        self.fc7 = nn.Linear(hidden_dim, 1).to(torch.float64)
        # self.fc7 = nn.Linear(hidden_dim, hidden_dim).to(torch.float64)
        self.relu = nn.ReLU().to(torch.float64)
        # self.softplus = nn.Softplus().to(torch.float64)
        self.softmax = nn.Softmax().to(torch.float64)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        x = self.relu(x)
        x = self.fc4(x)
        x = self.relu(x)
        x = self.fc5(x)
        x = self.relu(x)
        x = self.fc6(x)
        x = self.relu(x)
        x = self.fc7(x)
        return x

class Trend_Feature_model(nn.Module):
  def __init__(self, feature_dim, feature_hidden, lookback_dim, hidden_dim, output_dim, trend_weight = 0.4):
    super(Trend_Feature_model, self).__init__()
    self.trends_model = TrendsModel(lookback_dim, hidden_dim, output_dim)
    self.features_model = FeaturesModel(feature_dim, feature_hidden, output_dim)
    self.combining_layer = nn.Linear(feature_hidden + hidden_dim, hidden_dim).to(torch.float64)
    self.output_layer = nn.Linear(hidden_dim, 1).to(torch.float64)
    self.trend_weight = trend_weight
    self.relu = nn.ReLU().to(torch.float64)
    self.lookback_dim = lookback_dim
    self.hidden_dim = hidden_dim
  def forward(self, x):
    x_features = x[:,:-1 * self.lookback_dim]
    x_lookback = x[:,-1 * self.lookback_dim:]
    std_hour = x[:,-self.lookback_dim:].std(dim = 1).view(-1,1)
    feature_output = self.features_model(x_features)
    trends_output = self.trends_model(x_lookback)
    x = (feature_output * (1 - self.trend_weight)) + (trends_output * self.trend_weight)
    # x = torch.concat([feature_output,trends_output], axis = 1).to(torch.float64)
    # print(x.shape, self.hidden_dim)
    # x = self.relu(x)
    # x = self.combining_layer(x)
    # x = self.relu(x)
    # x = self.output_layer(x)

    vals = []
    for i in range(0, 11):
      q = i/10 if i not in [0,10] else (0.01 if i == 0 else 0.99)

      pred2 = x + (1 * std_hour) * torch.sqrt(torch.tensor(2)) * torch.erfinv(torch.tensor((2 * q) - 1))

      vals.append(pred2)

    x = torch.concat(vals, dim = 1)#.detach().numpy()

    return x



# Loss Functions

In [299]:
def crps_loss(y_pred, y_obs):
    n = len(y_pred)
    fx = y_pred
    fx_prob = torch.tensor([i for i in range(0,110,10)]).unsqueeze(0).repeat(y_pred.shape[0],1)
    # extend CDF min to ensure obs within forecast support
    # fx.shape = (n, d) ==> (n, d + 1)
    fx_min = torch.minimum(y_obs, y_pred[:, 0])
    y_pred = torch.hstack([fx_min[:, None], y_pred])
    fx_prob = torch.hstack([torch.zeros([n, 1]), fx_prob])

    # extend CDF max to ensure obs within forecast support
    # fx.shape = (n, d + 1) ==> (n, d + 2)
    idx = (fx[:, -1] < y_obs)
    fx_max = torch.maximum(y_obs, y_pred[:, -1])
    y_pred = torch.hstack([y_pred, fx_max[:, None]])
    fx_prob = torch.hstack([fx_prob, torch.full([n, 1], 100)])

    # indicator function:
    # - left of the obs is 0.0
    # - obs and right of the obs is 1.0
    o = torch.where(y_pred >= y_obs[:, None], 1.0, 0.0)

    # correct behavior when obs > max fx:
    # - should be 0 over range: max fx < x < obs
    o[idx, -1] = 0.0

    # forecast probabilities [unitless]
    f = fx_prob / 100.0
    # integrate along each sample, then average all samples
    crps = torch.mean(torch.trapezoid((f - o) ** 2, x=y_pred, axis=1))
    return crps# + torch.abs(((y_pred[:,5] - y_obs)) ** 2).mean()


def negLL(y_pred, y_true):
    mu = y_pred[:,6]
    prob_0 = y_pred[:,0]
    prob_100 = y_pred[:,-1]
    # print("y_pred.shape",y_pred.shape,"y_true.shape",y_true.shape)
    # sigma = y_pred[:,1]
    # denom = torch.sqrt(1/(2 * 2 * torch.pi))
    loss = (((mu - y_true)/ 2) ** 2) #- torch.log(denom)
    loss += 0.1 * (((prob_0 - y_true)/ 2) ** 2)
    loss += 0.1 * (((prob_100 - y_true)/ 2) ** 2)
    return loss.mean()

def negLL2(y_pred, y_true):
    # mu = y_pred[:,0]
    num_quantiles = 11
    obs =  y_true.unsqueeze(1).repeat(1,num_quantiles)
    # sigma = y_pred[:,1]
    # denom = torch.sqrt(1/(2 * 2 * torch.pi))
    loss = (((y_pred - obs)/ 2) ** 2) #- torch.log(denom)
    return loss.mean()

def mae(y_pred, y_true):
    mu = y_pred[:,0]
    # sigma = y_pred[:,1]
    # denom = torch.sqrt(1/(2 * 2 * torch.pi))
    loss = torch.abs(((mu - y_true))) #- torch.log(denom)
    return loss.mean()

def mae2(y_pred, y_true):
    # mu = y_pred[:,0]
    num_quantiles = 11
    obs =  y_true.unsqueeze(1).repeat(1,num_quantiles)
    # sigma = y_pred[:,1]
    # denom = torch.sqrt(1/(2 * 2 * torch.pi))
    loss = torch.abs(((y_pred - obs)/ 2)) #- torch.log(denom)
    return loss.mean()

#Dataloader

In [300]:

class CDataset(Dataset):
    def __init__(self, X, y):
        self.X = X#.values
        self.y = y#.values

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        x = self.X[idx]
        y = self.y[idx]
        return x, y

# Training Loop

In [ ]:
# from torch._C import float64
# Define the custom data loaders
batch_size = 64
train_dataset = CDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataset = CDataset(X_val, y_val)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

# Initialize the model
feature_dim = X_train.shape[1] - lookback_dim#22  # Specify the input feature dimensions
feature_hidden = 16
lookback_dim = lookback_dim
lookback_hidden = lookback_dim - 2

model = Trend_Feature_model(feature_dim = feature_dim, feature_hidden = int(feature_dim * 0.8),
                            lookback_dim=lookback_dim,
                            hidden_dim = lookback_hidden,
                            output_dim = 1,
                            trend_weight = 0.8)
# Define the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adam optimizer

# Training loop
num_epochs = 150  # Specify the number of training epochs
train_losses = []
val_losses = []

for epoch in range(num_epochs):

    model.eval()  # Set the model to evaluation mode
    val_loss = 0

    with torch.no_grad():
        for batch_X, batch_y in val_loader:

            outputs = model(batch_X)
            # loss = crps_loss(outputs, batch_y) + negLL(outputs, batch_y)
            # loss = negLL(outputs, batch_y)
            # loss = mae(outputs, batch_y)
            loss = crps_loss(outputs, batch_y)
            val_loss += loss.item()
    val_loss /= len(val_loader)
    val_losses.append(val_loss)

    model.train()  # Set the model to training mode
    i = 0
    train_loss = 0


    for batch_X, batch_y in train_loader:#range(3):
        # Forward pass
        outputs = model(batch_X)
        # loss = crps_loss(outputs, batch_y) + negLL(outputs, batch_y)
        loss = crps_loss(outputs, batch_y)
        # loss = negLL(outputs, batch_y)
        i += 1
        train_loss += loss.item()
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    train_losses.append(train_loss/i)
    # Validation loop

    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {loss.item()}, Val Loss: {val_loss}')


Epoch [1/150], Train Loss: 0.0573524365908368, Val Loss: 0.573455200303333
Epoch [2/150], Train Loss: 0.07146366477151554, Val Loss: 0.08098244077430729
Epoch [3/150], Train Loss: 0.05751769514033685, Val Loss: 0.06391580558010515
Epoch [4/150], Train Loss: 0.03871744699594345, Val Loss: 0.05745282084477952
Epoch [5/150], Train Loss: 0.042003058522924096, Val Loss: 0.05143763284232963
Epoch [6/150], Train Loss: 0.04276755870976003, Val Loss: 0.05386278507511725
Epoch [7/150], Train Loss: 0.032234577489935565, Val Loss: 0.04962717250772442
Epoch [8/150], Train Loss: 0.051853990072162286, Val Loss: 0.04699224763439578
Epoch [9/150], Train Loss: 0.038946257686905586, Val Loss: 0.05217568357126801
Epoch [10/150], Train Loss: 0.0383701263914414, Val Loss: 0.04565009698950046
Epoch [11/150], Train Loss: 0.032667068802251426, Val Loss: 0.04501219060074713
Epoch [12/150], Train Loss: 0.03903885407277077, Val Loss: 0.050228571510455296
Epoch [13/150], Train Loss: 0.044550849063010496, Val Loss:

In [ ]:
# outputs.shape, batch_y.shape

In [ ]:
plt.plot(train_losses, label = 'training loss')
plt.plot(val_losses, label = 'validation loss')
plt.legend()
plt.show()

# Inspection Space

In [ ]:
weather_cols = ['dewpoint_2m','relativehumidity_2m','temperature_2m',
                'apparent_temperature', 'precipitation', 'cloudcover', 'cloudcover_low',
                'cloudcover_mid', 'cloudcover_high', 'shortwave_radiation',
                'direct_radiation', 'diffuse_radiation', 'direct_normal_irradiance']
weather_dictionary = {col:[] for col in weather_cols}

In [ ]:
daily_crps = []
daily_crpss = []
for day in range(18, 29):

  start_date = '2023-06-{:02d}'.format(day - 1)
  end_date = '2023-06-{:02d}'.format(day)
  test_df = test_data[(test_data.time >= start_date + ' 01:00:00')
                    & (test_data.time <= end_date + ' 00:00:00')].drop(columns = 'time')

  X_test = test_df[[col for col in test_df.columns if col != 'value']]
  y_test = test_df.value
  X_test = torch.from_numpy(X_test.values).to(torch.float64)
  y_test = torch.from_numpy(y_test.values).to(torch.float64)

  preds = model(X_test)

  hourly_deviations = []
  meanies = []
  std_hour = X_test[:,-lookback_dim:].std(dim = 1).view(24,1)


  vals = []
  for i in range(0, 11):
    # q = i/10 if i not in [0,10] else (0.01 if i == 0 else 0.99)

    pred2 = preds[:,i] #+ (1 * std_hour) * torch.sqrt(torch.tensor(2)) * torch.erfinv(torch.tensor((2 * q) - 1))

    vals.append(pred2)
    plt.plot(pred2.detach().numpy(), label = str(i))

  plt.plot(y_test, color = 'black', label = 'real')
  plt.legend()
  plt.show()

  for col in weather_cols:
    weather_data = test_df[col].values
    weather_dictionary[col].append(test_df[col].std())


  pred = preds.detach().numpy()#torch.concat(vals, dim = 1).detach().numpy()

  # submission = pd.DataFrame(columns = ['timestamp']+['p' + str(i*10) for i in range(11)])
  # submission['timestamp'] = pd.date_range(start = '2023-06-{0:2d} 01:00:00'.format(day - 1),
  #                                         end =  '2023-06-{0:2d} 00:00:00'.format(day),
  #                                         freq='H',
  #                                         tz = states['ga']['local_time_zone']
  #                                         )

  # submission[['p' + str(i*10) for i in range(11)]] = pred



  obs = y_test.detach().numpy()
  fx_prob = torch.tensor([i for i in range(0,110,10)]).unsqueeze(0).repeat(24,1).numpy()
  crps, crpss, _ = crps_and_crpss_scores(obs, pred, start_date, end_date, state_location)
  print(crps, crpss)
  daily_crps.append(crps)
  daily_crpss.append(crpss)
  # submission.to_csv("/content/drive/MyDrive/net_load_forecasting/submissions/_new_model_June_{:02d}.csv".format(day))

# plt.plot(,daily_crps)
# plt.show()

In [ ]:
plt.scatter(range(len(daily_crps)), daily_crps)
plt.scatter(range(len(daily_crpss)), daily_crpss)
plt.axhline(0)
plt.show()

In [ ]:
day = 23
test_df = test_data[(test_data.time >= '2023-06-{:02d} 01:00:00'.format(day - 1)) & (test_data.time <= '2023-06-{:02d} 00:00:00'.format(day))].drop(columns = 'time')
plt.figure(figsize = (15,5))
for col in weather_cols:
  weather_data = test_df[col].values
  weather_dictionary[col].append(test_df[col].std())
  plt.plot(weather_data / np.max(np.abs(weather_data)), label = col)

plt.legend()
plt.show()

In [ ]:
day = 24
test_df = test_data[(test_data.time >= '2023-06-{:02d} 01:00:00'.format(day - 1)) & (test_data.time <= '2023-06-{:02d} 00:00:00'.format(day))].drop(columns = 'time')
plt.figure(figsize = (15,5))
for col in weather_cols:
  weather_data = test_df[col].values
  plt.plot(weather_data / np.max(np.abs(weather_data)), label = col)

plt.legend()
plt.show()

In [ ]:
# current_day = -1
# biases = []
# predictions = []
# observations = []
# for i in range(lookback_start, lookback_end + 1):
#   test_df = test_data[(-24 * (i) + 1):(-24 * (i) + 1) + 24].drop(columns = 'time')
#   X_test = test_df[[col for col in test_df.columns if col != 'value']]
#   y_test = test_df.value
#   X_test = torch.from_numpy(X_test.values).to(torch.float64)
#   y_test = torch.from_numpy(y_test.values).to(torch.float64)

#   preds = model(X_test)

#   predictions.append(preds)
#   observations.append(y_test)

# mean_pred = torch.zeros((24,1))
# mean_obs = torch.zeros((24,1))

# for i in range(lookback_dim):
#   mean_pred += predictions[i]
#   mean_obs += observations[i].view(24,1)

# mean_obs /= lookback_dim
# mean_pred /= lookback_dim

# test_df = test_data[-23-24:-23].drop(columns = 'time')
# X_test = test_df[[col for col in test_df.columns if col != 'value']]
# y_test = test_df.value
# X_test = torch.from_numpy(X_test.values).to(torch.float64)
# y_test = torch.from_numpy(y_test.values).to(torch.float64)

# preds = model(X_test)

# hourly_deviations = []
# meanies = []
# std_hour = X_test[:,-lookback_dim:].std(dim = 1).view(24,1)


# vals = []
# for i in range(0, 11):
#   q = i/10 if i not in [0,10] else (0.01 if i == 0 else 0.99)
#   bias = mean_pred - mean_obs
#   corrected_mean = preds - bias


#   pred2 = preds + std_hour * torch.sqrt(torch.tensor(2)) * torch.erfinv(torch.tensor((2 * q) - 1))

#   vals.append(pred2)
#   plt.plot(pred2.detach().numpy(), label = str(i))

# plt.legend()
# plt.show()

# pred = torch.concat(vals, dim = 1).detach().numpy()


# BIAS CORRECTION AND SUBMISSION

In [ ]:
test_df = test_data[-23-24-24:-23-24]#.drop(columns = 'time')

In [ ]:
current_day = -1
biases = []
predictions = []
observations = []
for i in range(lookback_start, lookback_end + 1):
  test_df = test_data[(-24 * (i) + 1):(-24 * (i) + 1) + 24].drop(columns = 'time')
  X_test = test_df[[col for col in test_df.columns if col != 'value']]
  y_test = test_df.value
  X_test = torch.from_numpy(X_test.values).to(torch.float64)
  y_test = torch.from_numpy(y_test.values).to(torch.float64)

  preds = model(X_test)

  predictions.append(preds)
  observations.append(y_test)

mean_pred = torch.zeros((24,11))
mean_obs = torch.zeros((24,1))

for i in range(lookback_dim):
  mean_pred += predictions[i]
  mean_obs += observations[i].view(24,1)

mean_obs /= lookback_dim
mean_pred /= lookback_dim

In [ ]:
# j = 1
# trial_x, trial_y = X_test[-24 * (j): -24 * (j-1)], y_test[-24 * (j): -24 * (j-1)]

test_df = test_data[-23-24-24:-23-24].drop(columns = 'time')
X_test = test_df[[col for col in test_df.columns if col != 'value']]
y_test = test_df.value
X_test = torch.from_numpy(X_test.values).to(torch.float64)
y_test = torch.from_numpy(y_test.values).to(torch.float64)

preds = model(X_test)#torch.from_numpy(X).to(torch.float64))#.shape

hourly_deviations = []
meanies = []
std_hour = X_test[:,-lookback_dim:].std(dim = 1).view(24,1)#new_df[new_df['hour'] == float(i)]['value'].std()


vals = []
# for i in range(0, 11):
#   q = i/10 if i not in [0,10] else (0.01 if i == 0 else 0.99)
#   bias = mean_pred - mean_obs
#   corrected_mean = preds - bias#(preds - mean)


#   pred2 = preds +  (2 * std_hour) * torch.sqrt(torch.tensor(2)) * torch.erfinv(torch.tensor((2 * q) - 1))

#   vals.append(pred2)
plt.plot(preds.detach().numpy(), label = str(i))

# plt.legend()
# plt.show()

pred = preds.detach().numpy()#torch.concat(vals, dim = 1).detach().numpy()

# Submission Block

In [256]:
from datetime import datetime


In [260]:
submission = pd.DataFrame(columns = ['timestamp']+['p' + str(i*10) for i in range(11)])
submission['timestamp'] = pd.date_range(start = '2023-07-04 01:00:00',
                                         end =  '2023-07-05 00:00:00',
                                         freq='H',
                                         tz = states[state_location]['local_time_zone']
                                         )
submission[['p' + str(i*10) for i in range(11)]] = pred

submission.to_csv("/content/drive/MyDrive/net_load_forecasting/submissions/{}_pytorch2_submission_{}.csv".format(state_location,datetime.now()))

upload_function(submission.set_index('timestamp'),state_location)